# Delta Lake 2.3.0 Release

In [1]:
import pyspark
from delta import *
from pyspark.sql import functions as F

import pathlib

builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-332-delta-230/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e4c7e6fa-948b-4695-9392-af98ccb0dc79;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 102ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |  

23/04/10 13:27:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
cwd = pathlib.Path().resolve()

## Convert Iceberg to Delta Lake

TODO: Figure out how to create an Iceberg table

## Delta Lake Shallow Clone

In [3]:
data = [(0, "Bob", 23), (1, "Sue", 25), (2, "Jim", 27)]

In [4]:
df = spark.createDataFrame(data).toDF("id", "name", "age")

In [6]:
df.write.format("delta").save("tmp/my_people")

23/04/10 13:28:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
spark.sql(f"CREATE TABLE delta.`{cwd}/tmp/my_cloned_people` SHALLOW CLONE delta.`{cwd}/tmp/my_people`")

DataFrame[source_table_size: bigint, source_num_of_files: bigint, num_removed_files: bigint, num_copied_files: bigint, removed_files_size: bigint, copied_files_size: bigint]

In [9]:
spark.read.format("delta").load("tmp/my_cloned_people").show()

+---+----+---+
| id|name|age|
+---+----+---+
|  2| Jim| 27|
|  1| Sue| 25|
|  0| Bob| 23|
+---+----+---+



In [10]:
!tree tmp/my_cloned_people

tmp/my_cloned_people
└── _delta_log
    ├── 00000000000000000000.checkpoint.parquet
    ├── 00000000000000000000.json
    └── _last_checkpoint

1 directory, 3 files


In [11]:
!tree tmp/my_people

tmp/my_people
├── _delta_log
│   └── 00000000000000000000.json
├── part-00000-61eded6b-5480-41b3-834b-e481e52b06bf-c000.snappy.parquet
├── part-00003-962d94a2-b9fa-4253-aaf5-5968eefd540c-c000.snappy.parquet
├── part-00006-8442af38-8b11-47fd-9a55-fa72f36d02b2-c000.snappy.parquet
└── part-00009-b5aa702a-d6fb-406e-97bf-f8f18b864f0a-c000.snappy.parquet

1 directory, 5 files


In [12]:
data = [(0, "Clare", 5),]

In [13]:
df = spark.createDataFrame(data).toDF("id", "name", "age")

In [14]:
df.write.format("delta").mode("append").save("tmp/my_people")

In [15]:
spark.read.format("delta").load("tmp/my_people").show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  0|Clare|  5|
|  1|  Sue| 25|
|  0|  Bob| 23|
|  2|  Jim| 27|
+---+-----+---+



In [16]:
spark.read.format("delta").load("tmp/my_cloned_people").show()

+---+----+---+
| id|name|age|
+---+----+---+
|  2| Jim| 27|
|  1| Sue| 25|
|  0| Bob| 23|
+---+----+---+



In [17]:
data = [(0, "Linda", 32),]

In [18]:
df = spark.createDataFrame(data).toDF("id", "name", "age")

In [19]:
df.write.format("delta").mode("append").save("tmp/my_cloned_people")

In [20]:
spark.read.format("delta").load("tmp/my_cloned_people").show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  0|Linda| 32|
|  2|  Jim| 27|
|  1|  Sue| 25|
|  0|  Bob| 23|
+---+-----+---+



## When not matched by source

## CREATE TABLE LIKE

In [10]:
df = spark.createDataFrame(
    [(0, "Bob", "Loblaw", 23), (1, "Sue", "Grafton", None), (2, "Jim", "Carrey", 61)]
).toDF("id", "first_name", "last_name", "age")

In [11]:
df.write.format("delta").saveAsTable(
    "default.famous_people"
)

In [12]:
spark.sql("select * from famous_people").show()

+---+----------+---------+----+
| id|first_name|last_name| age|
+---+----------+---------+----+
|  2|       Jim|   Carrey|  61|
|  0|       Bob|   Loblaw|  23|
|  1|       Sue|  Grafton|null|
+---+----------+---------+----+



In [13]:
spark.sql("CREATE TABLE famous_people_duplicate like famous_people")

DataFrame[]

In [14]:
spark.sql("select * from famous_people_duplicate").show()

+---+----------+---------+---+
| id|first_name|last_name|age|
+---+----------+---------+---+
+---+----------+---------+---+



## Reading change data feed in SQL queries

## Cleanup